# 08: Develop WBGT in the sun estimates
*Combine the solar radiation and wind values from `07_solar_radiation_wind.ipynb` to develop an estimate of how much higher WBGT would be in the sun as compared to the shade at any given day and location. Apply those time- and space-varying adjstments to the WBGT in the shade estimates developed in `06_bias_correction.ipynb` to create estimates of WBGT in the sun.*

In [ ]:
import warnings

import coiled
import numpy as np
import pandas as pd
import statsmodels.api as sm
import xarray as xr
from sklearn import linear_model
from utils import clean_up_times, gcm_list

warnings.filterwarnings("ignore")

Set up cluster to handle multiprocessing using a Dask client.

In [ ]:
cluster = coiled.Cluster(
    n_workers=1,
    name="08",
    worker_vm_types=["m7g.medium"],
    scheduler_vm_types=["m7g.medium"],
    region="us-west-2",
    spot_policy="spot_with_fallback",
)

cluster.adapt(minimum=1, maximum=100)

client = cluster.get_client()

In [ ]:
def shade_sun_adjustment_model():
    """
    Create linear model of the approximate difference between WBGT in the sun
    and WBGT in the shade, as derived from Figure S12 in Kong and Huber (2022).
    """
    X = np.array(
        [
            [300, 0.5],
            [500, 0.5],
            [700, 0.5],
            [900, 0.5],
            [300, 1],
            [500, 1],
            [700, 1],
            [900, 1],
            [300, 2],
            [500, 2],
            [700, 2],
            [900, 2],
            [300, 3],
            [500, 3],
            [700, 3],
            [900, 3],
        ]
    )
    y = np.array(
        [-3, -5, -6, -7, -2, -3.5, -4.5, -6, -1.5, -2.5, -3.5, -4.5, -1.5, -2, -3, -3.5]
    )
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    X = sm.add_constant(X)
    return sm.OLS(y, X).fit()

Set up a daily and subdaily template dataframe to use for all of the cities.

In [ ]:
def calc_wbgt_in_the_sun(wbgt_shade, adjustment_model, rsds, wind):
    """
    Given maximum daily solar radiation and daily average wind, use the defined
    WBGT adjustment model to calculate a time-varying adjustment. Apply that adjustment
    to timeseries of WBGT in the shade to create timeseries of WBGT in the sun.
    """
    df_daily = pd.DataFrame(index=pd.date_range("1985-01-01", "2099-12-31")).drop(
        pd.date_range("2015-01-01", "2019-12-31")
    )
    # Adjust maximum radiation by 0.75 per Parsons et al (2021) to account for the
    # fact that WBGT typically peaks a few hours after solar radiation peaks, when
    # the solar radiation is ~75% of the peak
    radiation_adjustment = 0.75
    df_daily["rsds_max"] = rsds * radiation_adjustment

    df_daily["sfcWind"] = wind
    # where wind > 3 or < 0.5 clip to constrain to domain of the adjustment model
    df_daily["sfcWind_adjusted"] = df_daily["sfcWind"].where(df_daily["sfcWind"] < 3, 3)
    df_daily["sfcWind_adjusted"] = df_daily["sfcWind_adjusted"].where(
        df_daily["sfcWind_adjusted"] > 0.5, 0.5
    )
    # where solar rad > 900 or < 300 clip to constrain to domain of the adjustment model
    df_daily["rsds_max_adjusted"] = df_daily["rsds_max"].where(
        df_daily["rsds_max"] < 900, 900
    )
    df_daily["rsds_max_adjusted"] = df_daily["rsds_max_adjusted"].where(
        df_daily["rsds_max_adjusted"] > 300, 300
    )
    try:
        df_daily["shade_sun_adjustment"] = adjustment_model.predict(
            sm.add_constant(df_daily[["rsds_max_adjusted", "sfcWind_adjusted"]].values)
        )
    except ValueError:
        df_daily["shade_sun_adjustment"] = np.nan
    wbgt_sun = wbgt_shade - df_daily["shade_sun_adjustment"].values
    return wbgt_sun

Load the model to calculate how much cooler it is in the shade compared to the sun.

In [ ]:
adjustment_model = shade_sun_adjustment_model()

Load wind, solar radiation, and WBGT in the shade estimates and create corresponding WBGT in the sun estimates.

In [ ]:
for gcm in gcm_list[0:1]:
    for scenario in ["ssp245", "ssp370"]:
        ds_historical = xr.open_zarr(
            f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-regions/"
            f"{gcm}-historical-bc.zarr"
        )
        ds_2030 = xr.open_zarr(
            f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-regions/"
            f"{gcm}-{scenario}-2030-bc.zarr"
        )
        ds_2050 = xr.open_zarr(
            f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-regions/"
            f"{gcm}-{scenario}-2050-bc.zarr"
        )
        ds_2090 = xr.open_zarr(
            f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-regions/"
            f"{gcm}-{scenario}-2090-bc.zarr"
        ).sel(time=slice("2090-01-01", "2099-12-31"))

        wbgt_shade = xr.concat(
            [ds_historical, ds_2030, ds_2050, ds_2090], dim="time"
        ).chunk({"time": -1, "processing_id": 850})
        wind = (
            clean_up_times(
                xr.open_zarr(
                    f"s3://carbonplan-scratch/extreme-heat/wind_solrad-regions/{gcm}-{scenario}-"
                    "wind-solrad-regions.zarr"
                )[["sfcWind"]].sel(
                    processing_id=wbgt_shade["scen"].processing_id.values
                )
            )
            .chunk({"time": -1, "processing_id": 850})
            .sel(time=slice("1985-01-01", "2099-12-31"))
        )
        wind["time"] = wind["time"].dt.floor("D")

        rsds = clean_up_times(
            xr.open_zarr(
                f"s3://carbonplan-scratch/extreme-heat/wind_solrad-regions/{gcm}-{scenario}-"
                "rsds-max-regions.zarr"
            )
            .rename({"__xarray_dataarray_variable__": "rsds"})[["rsds"]]
            .sel(processing_id=wbgt_shade["scen"].processing_id.values)
            .sel(time=slice("1985-01-01", "2099-12-31"))
        ).chunk({"time": -1, "processing_id": 850})
        rsds["time"] = rsds["time"].dt.floor("D")

        out = xr.apply_ufunc(
            calc_wbgt_in_the_sun,
            wbgt_shade["scen"],
            adjustment_model,
            rsds["rsds"],
            wind["sfcWind"],
            input_core_dims=[["time"], [], ["time"], ["time"]],
            output_core_dims=[["time"]],
            vectorize=True,
            dask="parallelized",
            output_dtypes=[wbgt_shade["scen"].dtype],
        )
        out = out.to_dataset(name="WBGT-sun").chunk({"processing_id": 850, "time": -1})
        out = out.to_zarr(
            f"s3://carbonplan-scratch/extreme-heat/wbgt-sun-regions/wbgt-sun-{gcm}-{scenario}.zarr",
            mode="w",
            consolidated=True,
        )